In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("C:\\Users\\Smruti Deshpande\\Desktop\\Projects\\Datathon_kj\\elearning_churn_dataset.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   User ID                        1000 non-null   object 
 1   Enrollment Date                1000 non-null   object 
 2   Subscription Plan              1000 non-null   object 
 3   Course Completion Rate         1000 non-null   float64
 4   Courses Enrolled               1000 non-null   int64  
 5   Login Frequency (per month)    1000 non-null   int64  
 6   Time Spent per Session (mins)  1000 non-null   float64
 7   Last Active Date               1000 non-null   object 
 8   Forum Participation            1000 non-null   int64  
 9   Quiz Participation Rate        1000 non-null   float64
 10  Assignment Submission Rate     1000 non-null   float64
 11  Video Watch Completion Rate    1000 non-null   float64
 12  Survey Response (1-5)          1000 non-null   in

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Handle Missing Values
# For numerical columns, we fill with median or mean based on the nature of the column
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_cols] = df[numerical_cols].apply(lambda x: x.fillna(x.median()))

# For categorical columns, we fill missing values using forward fill or backward fill
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].apply(lambda x: x.fillna(method='ffill'))

# Handle Duplicates
df = df.drop_duplicates()

# Feature Engineering
# Engagement Score Calculation
df['Engagement Score'] = df['Login Frequency (per month)'] * df['Time Spent per Session (mins)']

# Encode Categorical Features
encoder = LabelEncoder()
df['Subscription Plan'] = encoder.fit_transform(df['Subscription Plan'])
df['Feedback Sentiment'] = encoder.fit_transform(df['Feedback Sentiment'])
df['Support Ticket Sentiment'] = encoder.fit_transform(df['Support Ticket Sentiment'])
df['Course Reminder Response'] = encoder.fit_transform(df['Course Reminder Response'])
df['Gamified Progress Engagement'] = encoder.fit_transform(df['Gamified Progress Engagement'])

# Convert Date columns to datetime
df['Enrollment Date'] = pd.to_datetime(df['Enrollment Date'], format='%d-%m-%Y')
df['Last Active Date'] = pd.to_datetime(df['Last Active Date'], format='%d-%m-%Y')

# Calculate the days since last active
df['Days Since Last Active'] = (pd.to_datetime('today') - df['Last Active Date']).dt.days

# Normalize numerical features
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Separate Independent Variables and Target Variable
X = df.drop(['churn'], axis=1)  # Independent features
y = df['churn']  # Target variable


C:\Users\Smruti Deshpande\AppData\Local\Temp\ipykernel_8624\3177323329.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[categorical_cols] = df[categorical_cols].apply(lambda x: x.fillna(method='ffill'))


In [6]:
y = df['churn'].astype(int)  # Ensure the churn column is in integer format (0 or 1)

In [12]:
%pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 1.0/124.9 MB 10.1 MB/s eta 0:00:13
    --------------------------------------- 2.1/124.9 MB 4.9 MB/s eta 0:00:26
    --------------------------------------- 2.9/124.9 MB 4.7 MB/s eta 0:00:27
   - -------------------------------------- 3.4/124.9 MB 3.9 MB/s eta 0:00:32
   - -------------------------------------- 3.9/124.9 MB 3.6 MB/s eta 0:00:34
   - -------------------------------------- 4.7/124.9 MB 3.7 MB/s eta 0:00:33
   - -------------------------------------- 5.5/124.9 MB 3.8 MB/s eta 0:00:32
   -- ------------------------------------- 6.6/124.9 MB 3.8 MB/s eta 0:00:31
   -- ------------------------------------- 7.1/124.9 MB 3.7 MB/s eta 0:00:32
   -- ------------------------------------- 7.6/124.9 MB 3.6 MB/s eta 0:00:33
   -- ------------------------------------- 8.4/124.9 MB 3.6 MB/s eta 0:00:33
   -- ------------------------------------- 9.2/124.9 MB 3.6 MB/s eta 

In [13]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Convert the 'churn' column to binary if necessary
y = df['churn'].astype(int)

# Features to use for churn prediction
X_churn = df[['Subscription Plan', 'Course Completion Rate', 'Survey Response (1-5)', 'Time Spent per Session (mins)',
              'Assignment Submission Rate', 'Video Watch Completion Rate', 'Engagement Score']]

# Encode categorical variables (if necessary)
X_churn['Subscription Plan'] = X_churn['Subscription Plan'].astype('category').cat.codes

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_churn, y, test_size=0.2, random_state=42)

# Train the XGBoost model
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Predictions
y_pred = xgb_model.predict(X_test)

# Evaluation
print("XGBoost Classifier - Churn Prediction")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


C:\Users\Smruti Deshpande\AppData\Local\Temp\ipykernel_8624\1414443987.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_churn['Subscription Plan'] = X_churn['Subscription Plan'].astype('category').cat.codes


XGBoost Classifier - Churn Prediction
[[117  19]
 [ 44  20]]
              precision    recall  f1-score   support

           0       0.73      0.86      0.79       136
           1       0.51      0.31      0.39        64

    accuracy                           0.69       200
   macro avg       0.62      0.59      0.59       200
weighted avg       0.66      0.69      0.66       200



In [20]:
import joblib

# Save the trained XGBoost model
joblib.dump(xgb_model, 'xgb_churn_model.pkl')

# Load the XGBoost model
loaded_model = joblib.load('xgb_churn_model.pkl')

# Use the loaded model to make predictions
y_pred_loaded = loaded_model.predict(X_test)

# Evaluation
print("Loaded XGBoost Classifier - Churn Prediction")
print(confusion_matrix(y_test, y_pred_loaded))
print(classification_report(y_test, y_pred_loaded))


Loaded XGBoost Classifier - Churn Prediction
[[117  19]
 [ 44  20]]
              precision    recall  f1-score   support

           0       0.73      0.86      0.79       136
           1       0.51      0.31      0.39        64

    accuracy                           0.69       200
   macro avg       0.62      0.59      0.59       200
weighted avg       0.66      0.69      0.66       200



In [15]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Convert the 'churn' column to binary if necessary
y = df['churn'].astype(int)

# Features to use for churn prediction
X_churn = df[['Subscription Plan', 'Course Completion Rate', 'Survey Response (1-5)', 'Time Spent per Session (mins)',
              'Assignment Submission Rate', 'Video Watch Completion Rate', 'Engagement Score']]

# Encode categorical variables (if necessary)
X_churn['Subscription Plan'] = X_churn['Subscription Plan'].astype('category').cat.codes

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_churn, y, test_size=0.2, random_state=42)

# Train the CatBoost model
catboost_model = CatBoostClassifier(iterations=100, random_state=42, verbose=0)
catboost_model.fit(X_train, y_train)

# Predictions
y_pred = catboost_model.predict(X_test)

# Evaluation
print("CatBoost Classifier - Churn Prediction")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


C:\Users\Smruti Deshpande\AppData\Local\Temp\ipykernel_8624\3484681465.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_churn['Subscription Plan'] = X_churn['Subscription Plan'].astype('category').cat.codes


CatBoost Classifier - Churn Prediction
[[133   3]
 [ 62   2]]
              precision    recall  f1-score   support

           0       0.68      0.98      0.80       136
           1       0.40      0.03      0.06        64

    accuracy                           0.68       200
   macro avg       0.54      0.50      0.43       200
weighted avg       0.59      0.68      0.57       200



In [25]:
import joblib
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np

# Convert the 'churn' column to binary if necessary
y = df['churn'].astype(int)

# Features to use for churn prediction
X_churn = df[['Subscription Plan', 'Course Completion Rate', 'Survey Response (1-5)', 'Time Spent per Session (mins)',
              'Assignment Submission Rate', 'Video Watch Completion Rate', 'Engagement Score']]

# Encode categorical variables (if necessary)
X_churn['Subscription Plan'] = X_churn['Subscription Plan'].astype('category').cat.codes

# Handle class imbalance using SMOTE (oversampling the minority class)
smote = SMOTE(random_state=42)
X_churn_resampled, y_resampled = smote.fit_resample(X_churn, y)

# Train-Test Split (we will use cross-validation for model evaluation instead)
X_train, X_test, y_train, y_test = train_test_split(X_churn_resampled, y_resampled, test_size=0.2, random_state=42)

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'scale_pos_weight': [1, 10]  # Adjust this to handle class imbalance
}

grid_search = GridSearchCV(estimator=xgb.XGBClassifier(random_state=42), param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters from GridSearchCV
print(f"Best parameters found: {grid_search.best_params_}")

# Train the XGBoost model using the best parameters
xgb_model = grid_search.best_estimator_

# 10-fold Cross-Validation to evaluate the model performance
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
accuracies = []

for train_idx, test_idx in skf.split(X_churn_resampled, y_resampled):
    X_train_cv, X_test_cv = X_churn_resampled.iloc[train_idx], X_churn_resampled.iloc[test_idx]
    y_train_cv, y_test_cv = y_resampled.iloc[train_idx], y_resampled.iloc[test_idx]

    # Fit the model on the current fold
    xgb_model.fit(X_train_cv, y_train_cv)
    
    # Make predictions
    y_pred_cv = xgb_model.predict(X_test_cv)
    
    # Calculate accuracy for the fold
    accuracies.append(accuracy_score(y_test_cv, y_pred_cv))

print(f"\nAverage Accuracy from 10-Fold Cross-Validation: {np.mean(accuracies):.4f}")

# Now, train the model on the full resampled dataset for final evaluation
xgb_model.fit(X_churn_resampled, y_resampled)

# Save the trained model using joblib
joblib.dump(xgb_model, 'xgb_churn_model.pkl')

# Load the model from the saved file
loaded_model = joblib.load('xgb_churn_model.pkl')

# Predictions with the loaded model
y_pred_loaded = loaded_model.predict(X_test)

# Evaluate model predictions
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_loaded))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_loaded))

# Accuracy Score
accuracy = accuracy_score(y_test, y_pred_loaded)
print(f"\nAccuracy: {accuracy:.4f}")

# Optional: Display some predicted vs actual values
predicted_vs_actual = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_loaded})
print("\nPredicted vs Actual (first 10 examples):")
print(predicted_vs_actual.head(10))


C:\Users\Smruti Deshpande\AppData\Local\Temp\ipykernel_8624\2139652118.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_churn['Subscription Plan'] = X_churn['Subscription Plan'].astype('category').cat.codes


Best parameters found: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'scale_pos_weight': 1}

Average Accuracy from 10-Fold Cross-Validation: 0.7275
Confusion Matrix:
[[130   6]
 [ 22 122]]

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.96      0.90       136
           1       0.95      0.85      0.90       144

    accuracy                           0.90       280
   macro avg       0.90      0.90      0.90       280
weighted avg       0.91      0.90      0.90       280


Accuracy: 0.9000

Predicted vs Actual (first 10 examples):
      Actual  Predicted
78         0          0
478        0          0
354        1          1
594        1          1
1255       1          1
993        0          0
254        0          0
567        1          0
316        1          0
1131       1          1


In [26]:
# Calculate the predicted churn rate
predicted_churns = sum(y_pred_loaded)  # Count the number of predicted churns (1's)
total_predictions = len(y_pred_loaded)  # Total number of predictions

# Churn Rate (as percentage)
churn_rate = (predicted_churns / total_predictions) * 100

print(f"Predicted Churn Rate: {churn_rate:.2f}%")


Predicted Churn Rate: 45.71%
